In [1]:
from video_chatgpt.video_conversation import conv_templates, SeparatorStyle
from video_chatgpt.model.utils import KeywordsStoppingCriteria
import torch
from tqdm import tqdm

#add new packages as below
from PIL import Image
from decord import VideoReader, cpu
import argparse
import numpy as np
import os

import os
import numpy as np
from PIL import Image
from decord import VideoReader, cpu
from transformers import AutoTokenizer, CLIPVisionModel, CLIPImageProcessor
from video_chatgpt.model import VideoChatGPTLlamaForCausalLM
from video_chatgpt.utils import disable_torch_init
from video_chatgpt.constants import *
import torch


# Define constants
DEFAULT_VIDEO_TOKEN = "<video>"
DEFAULT_VIDEO_PATCH_TOKEN = "<vid_patch>"
DEFAULT_VID_START_TOKEN = "<vid_start>"
DEFAULT_VID_END_TOKEN = "<vid_end>"

In [2]:
def load_video(vis_path, n_clips=1, num_frm=100):
    """
    Load video frames from a video file.

    Parameters:
    vis_path (str): Path to the video file.
    n_clips (int): Number of clips to extract from the video. Defaults to 1.
    num_frm (int): Number of frames to extract from each clip. Defaults to 100.

    Returns:
    list: List of PIL.Image.Image objects representing video frames.
    """

    # Load video with VideoReader
    vr = VideoReader(vis_path, ctx=cpu(0))
    total_frame_num = len(vr)

    # Currently, this function supports only 1 clip
    assert n_clips == 1

    # Calculate total number of frames to extract
    total_num_frm = min(total_frame_num, num_frm)
    # Get indices of frames to extract
    frame_idx = get_seq_frames(total_frame_num, total_num_frm)
    # Extract frames as numpy array
    img_array = vr.get_batch(frame_idx).asnumpy()
    # Set target image height and width
    target_h, target_w = 224, 224
    # If image shape is not as target, resize it
    if img_array.shape[-3] != target_h or img_array.shape[-2] != target_w:
        img_array = torch.from_numpy(img_array).permute(0, 3, 1, 2).float()
        img_array = torch.nn.functional.interpolate(img_array, size=(target_h, target_w))
        img_array = img_array.permute(0, 2, 3, 1).to(torch.uint8).numpy()

    # Reshape array to match number of clips and frames
    img_array = img_array.reshape(
        (n_clips, total_num_frm, img_array.shape[-3], img_array.shape[-2], img_array.shape[-1]))
    # Convert numpy arrays to PIL Image objects
    clip_imgs = [Image.fromarray(img_array[0, j]) for j in range(total_num_frm)]

    return clip_imgs


def get_seq_frames(total_num_frames, desired_num_frames):
    """
    Calculate the indices of frames to extract from a video.

    Parameters:
    total_num_frames (int): Total number of frames in the video.
    desired_num_frames (int): Desired number of frames to extract.

    Returns:
    list: List of indices of frames to extract.
    """

    # Calculate the size of each segment from which a frame will be extracted
    seg_size = float(total_num_frames - 1) / desired_num_frames

    seq = []
    for i in range(desired_num_frames):
        # Calculate the start and end indices of each segment
        start = int(np.round(seg_size * i))
        end = int(np.round(seg_size * (i + 1)))

        # Append the middle index of the segment to the list
        seq.append((start + end) // 2)

    return seq


def initialize_model(model_name, projection_path=None):
    """
    Initializes the model with given parameters.

    Parameters:
    model_name (str): Name of the model to initialize.
    projection_path (str, optional): Path to the projection weights. Defaults to None.

    Returns:
    tuple: Model, vision tower, tokenizer, image processor, vision config, and video token length.
    """

    # Disable initial torch operations
    disable_torch_init()

    # Convert model name to user path
    model_name = os.path.expanduser(model_name)

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load model
    model = VideoChatGPTLlamaForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16,
                                                         use_cache=True)
    
    # Load image processor
    # Fan Zheyu: the line below changed hf model name to path to avoid connection problem
    image_processor = CLIPImageProcessor.from_pretrained("/root/autodl-tmp/clip-vit-large-patch14", torch_dtype=torch.float16)

    # Set to use start and end tokens for video
    mm_use_vid_start_end = True

    # Add tokens to tokenizer
    tokenizer.add_tokens([DEFAULT_VIDEO_PATCH_TOKEN], special_tokens=True)
    if mm_use_vid_start_end:
        tokenizer.add_tokens([DEFAULT_VID_START_TOKEN, DEFAULT_VID_END_TOKEN], special_tokens=True)

    # Resize token embeddings of the model
    model.resize_token_embeddings(len(tokenizer))

    # Load the weights from projection_path after resizing the token_embeddings
    if projection_path:
        print(f"Loading weights from {projection_path}")
        status = model.load_state_dict(torch.load(projection_path, map_location='cpu'), strict=False)
        if status.unexpected_keys:
            print(f"Unexpected Keys: {status.unexpected_keys}.\nThe Video-ChatGPT weights are not loaded correctly.")
        print(f"Weights loaded from {projection_path}")

    # Set model to evaluation mode and move to GPU
    model = model.eval()
    model = model.cuda()

    # Fan Zheyu: the line below changed hf model name to path to avoid connection problem
    vision_tower_name = "/root/autodl-tmp/clip-vit-large-patch14"

    # Load vision tower and move to GPU
    vision_tower = CLIPVisionModel.from_pretrained(vision_tower_name, torch_dtype=torch.float16,
                                                   low_cpu_mem_usage=True).cuda()
    vision_tower = vision_tower.eval()

    # Configure vision model
    vision_config = model.get_model().vision_config
    vision_config.vid_patch_token = tokenizer.convert_tokens_to_ids([DEFAULT_VIDEO_PATCH_TOKEN])[0]
    vision_config.use_vid_start_end = mm_use_vid_start_end
    if mm_use_vid_start_end:
        vision_config.vid_start_token, vision_config.vid_end_token = tokenizer.convert_tokens_to_ids(
            [DEFAULT_VID_START_TOKEN, DEFAULT_VID_END_TOKEN])

    # Set video token length
    video_token_len = 356

    return model, vision_tower, tokenizer, image_processor, video_token_len


In [3]:
def get_spatio_temporal_features_torch(features):
    """
    Computes spatio-temporal features from given features.

    Parameters:
    features (torch.Tensor): Input features to process.

    Returns:
    torch.Tensor: Spatio-temporal features.
    """

    # Extract the dimensions of the features
    t, s, c = features.shape

    # Compute temporal tokens as the mean along the time axis
    temporal_tokens = torch.mean(features, dim=1)

    # Padding size calculation
    padding_size = 100 - t

    # Pad temporal tokens if necessary
    if padding_size > 0:
        padding = torch.zeros(padding_size, c, device=features.device)
        temporal_tokens = torch.cat((temporal_tokens, padding), dim=0)

    # Compute spatial tokens as the mean along the spatial axis
    spatial_tokens = torch.mean(features, dim=0)

    # Concatenate temporal and spatial tokens and cast to half precision
    concat_tokens = torch.cat([temporal_tokens, spatial_tokens], dim=0).half()

    return concat_tokens

In [4]:
def video_chatgpt_infer(video_frames, question, conv_mode, model, vision_tower, tokenizer, image_processor, video_token_len):
    """
    Run inference using the Video-ChatGPT model.

    Parameters:
    sample : Initial sample
    video_frames (torch.Tensor): Video frames to process.
    question (str): The question string.
    conv_mode: Conversation mode.
    model: The pretrained Video-ChatGPT model.
    vision_tower: Vision model to extract video features.
    tokenizer: Tokenizer for the model.
    image_processor: Image processor to preprocess video frames.
    video_token_len (int): The length of video tokens.

    Returns:
    dict: Dictionary containing the model's output.
    """

    # Prepare question string for the model
    if model.get_model().vision_config.use_vid_start_end:
        qs = question + '\n' + DEFAULT_VID_START_TOKEN + DEFAULT_VIDEO_PATCH_TOKEN * video_token_len + DEFAULT_VID_END_TOKEN
    else:
        qs = question + '\n' + DEFAULT_VIDEO_PATCH_TOKEN * video_token_len

    # Prepare conversation prompt
    conv = conv_templates[conv_mode].copy()
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    # Tokenize the prompt
    inputs = tokenizer([prompt])

    # Preprocess video frames and get image tensor
    image_tensor = image_processor.preprocess(video_frames, return_tensors='pt')['pixel_values']

    # Move image tensor to GPU and reduce precision to half
    image_tensor = image_tensor.half().cuda()

    # Generate video spatio-temporal features
    with torch.no_grad():
        image_forward_outs = vision_tower(image_tensor, output_hidden_states=True)
        frame_features = image_forward_outs.hidden_states[-2][:, 1:] # Use second to last layer as in LLaVA
    video_spatio_temporal_features = get_spatio_temporal_features_torch(frame_features)

    # Move inputs to GPU
    input_ids = torch.as_tensor(inputs.input_ids).cuda()

    # Define stopping criteria for generation
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    stopping_criteria = KeywordsStoppingCriteria([stop_str], tokenizer, input_ids)

    # Run model inference
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            video_spatio_temporal_features=video_spatio_temporal_features.unsqueeze(0),
            do_sample=True,
            temperature=0.2,
            max_new_tokens=1024,
            stopping_criteria=[stopping_criteria])

    # Check if output is the same as input
    n_diff_input_output = (input_ids != output_ids[:, :input_ids.shape[1]]).sum().item()
    if n_diff_input_output > 0:
        print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')

    # Decode output tokens
    outputs = tokenizer.batch_decode(output_ids[:, input_ids.shape[1]:], skip_special_tokens=True)[0]

    # Clean output string
    outputs = outputs.strip().rstrip(stop_str).strip()

    return outputs

In [5]:
def unirun(model_name, video_path, projection_path, conv_mode, model, vision_tower, tokenizer, image_processor, video_token_len, question):

    video_path = video_path

    if os.path.exists(video_path):
        video_frames = load_video(video_path)

    conv_mode = conv_mode

    try:
        # Run inference on the video and add the output to the list
        output = video_chatgpt_infer(video_frames, question, conv_mode, model, vision_tower,
                                            tokenizer, image_processor, video_token_len)
        return output
        
    except Exception as e:
        print(f"Error processing video file '{video_path}': {e}")

In [6]:
## initialize model

model_name = "/root/autodl-tmp/LLaVA-7B-Lightening-v1-1"
projection_path = "/root/autodl-tmp/Video-ChatGPT-7B/video_chatgpt-7B.bin"

model, vision_tower, tokenizer, image_processor, video_token_len = initialize_model(model_name, projection_path)

You are using a model of type llava to instantiate a model of type VideoChatGPT. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights from /root/autodl-tmp/Video-ChatGPT-7B/video_chatgpt-7B.bin
Weights loaded from /root/autodl-tmp/Video-ChatGPT-7B/video_chatgpt-7B.bin


Some weights of the model checkpoint at /root/autodl-tmp/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.7.self_attn.v_proj.weight', 'text_model.encoder.layers.9.self_attn.k_proj.weight', 'text_model.encoder.layers.11.self_attn.k_proj.weight', 'text_model.encoder.layers.1.layer_norm2.bias', 'text_model.encoder.layers.2.layer_norm1.weight', 'text_model.encoder.layers.9.layer_norm1.bias', 'text_model.encoder.layers.10.self_attn.out_proj.weight', 'text_model.encoder.layers.7.layer_norm2.weight', 'text_model.encoder.layers.1.self_attn.out_proj.bias', 'text_model.encoder.layers.1.self_attn.q_proj.bias', 'text_model.encoder.layers.2.self_attn.out_proj.weight', 'text_model.encoder.layers.10.mlp.fc2.bias', 'text_model.encoder.layers.3.self_attn.out_proj.bias', 'text_model.encoder.layers.10.layer_norm1.weight', 'text_model.encoder.layers.4.layer_norm1.bias', 'text_model.encoder.layers.6.self_attn.v_proj.bias', 'text_model.encoder.layers.3.mlp.

In [7]:
import json
from moviepy.editor import VideoFileClip
from bert_score import score

to_write_star = {"Interaction": list(), "Sequence": list(), "Prediction": list(), "Feasibility": list()}
to_write_gpt = []

with open("/root/Video-ChatGPT/STAR/Questions, Answers and Situation Graphs/STAR_test.json") as file:
    tests = json.load(file)

questions_find = {"Int": "Interaction", "Seq": "Sequence", "Pre": "Prediction", "Fea": "Feasibility"}

for ele in tqdm(tests):
    question = ele['question']
    video_path = "/root/autodl-tmp/Charades_v1_480/"+ele['video_id']+'.mp4'
    full_video_frames = load_video(video_path)
    duration = VideoFileClip(video_path).duration
    start_sec, end_sec = max(0, ele['start']), min(duration, ele['end'])
    start_idx = max(0, round(len(full_video_frames) * (start_sec/duration) - 1))
    end_idx = min(len(full_video_frames) - 1, round(len(full_video_frames) * (end_sec/duration) - 1))
    video_frams = full_video_frames[start_idx: end_idx+1]

    pred = unirun(model_name, video_path, projection_path, "video-chatgpt_v1", model, vision_tower, tokenizer, image_processor, video_token_len, question)
    
    scores = []
    for i in range(4):
        _, _, F1 = score([pred], [ele['choices'][i]['choice']], lang="en", verbose=False)
        scores.append(F1)
    pred_choice = scores.index(max(scores))
    to_write_star[questions_find[ele['question_id'][0:3]]].append({"question_id": ele['question_id'], "answer": pred_choice})
    

Some weights of the model checkpoint at /root/autodl-tmp/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  1%|          | 71/7377 [05:59<10:31:45,  5.19s/it]Some weights of the model checkpoint at /root/autodl-tmp/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   end_idx = min(len(full_video_frames) - 1, round(len(full_video_frames) * (end_sec/du    │
│   21 │   video_frams = full_video_frames[start_idx: end_idx+1]                                   │
│   22 │                                                                                           │
│ ❱ 23 │   pred = unirun(model_name, video_path, projection_path, "video-chatgpt_v1", model, vi    │
│   24 │                                                                                           │
│   25 │   scores = []                                                                             │
│   26 │   for i in range(4):                                                                      │
│                                                                                                  │
│ in unirun:6                                                                                      │
│                                                                                                  │
│    3 │   video_path = video_path                                                                 │
│    4 │                                                                                           │
│    5 │   if os.path.exists(video_path):                                                          │
│ ❱  6 │   │   video_frames = load_video(video_path)                                               │
│    7 │                                                                                           │
│    8 │   conv_mode = conv_mode                                                                   │
│    9                                                                                             │
│                                                                                                  │
│ in load_video:26                                                                                 │
│                                                                                                  │
│    23 │   # Get indices of frames to extract                                                     │
│    24 │   frame_idx = get_seq_frames(total_frame_num, total_num_frm)                             │
│    25 │   # Extract frames as numpy array                                                        │
│ ❱  26 │   img_array = vr.get_batch(frame_idx).asnumpy()                                          │
│    27 │   # Set target image height and width                                                    │
│    28 │   target_h, target_w = 224, 224                                                          │
│    29 │   # If image shape is not as target, resize it                                           │
│                                                                                                  │
│ /root/miniconda3/envs/videochatgpt/lib/python3.10/site-packages/decord/video_reader.py:175 in    │
│ get_batch                                                                                        │
│                                                                                                  │
│   172 │   │   """                                                                                │
│   173 │   │   assert self._handle is not None                                                    │
│   174 │   │   indices = _nd.array(self._validate_indices(indices))                               │
│ ❱ 175 │   │   arr = _CAPI_VideoReaderGetBatch(self._handle, indices)                             │
│   176 │   │   return bridge_out(arr)                                                             │
│   177 │                                                    

In [ ]:
try:
    with open(os.path.join("/root/Video-ChatGPT/", f"preds_star.json"), 'w') as file:
        json.dump(to_write_star, file)
except:
    try:
        with open(os.path.join("/root/Video-ChatGPT", f"preds_star.json"), 'w') as file:
            json.dump(to_write_star, file)
    except:
        try:
            with open(os.path.join("/root/Video-ChatGPT/", f"preds_star_2.json"), 'w') as file:
                json.dump(to_write_star, file)
        except:
            with open(os.path.join("/root/Video-ChatGPT", f"preds_star_2.json"), 'w') as file:
                json.dump(to_write_star, file)